In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, train_test_split, ParameterGrid
from sklearn.metrics import accuracy_score

In [2]:
data = pd.read_csv('data/significant-features-data.csv')
data.head()

,Age at enrollment,Curricular units 1st sem (evaluations),Curricular units 1st sem (approved),Curricular units 1st sem (grade),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Unemployment rate,Target,y_labels
0,0.056604,0.000000,0.000000,0.000000,0.0,0.00000,0.00,0.000000,0.372093,Dropout,-1
1,0.037736,0.133333,0.230769,0.741722,0.0,0.26087,0.30,0.735897,0.732558,Graduate,1
2,0.037736,0.000000,0.000000,0.000000,0.0,0.26087,0.00,0.000000,0.372093,Dropout,-1
3,0.056604,0.177778,0.230769,0.711447,0.0,0.26087,0.25,0.667692,0.209302,Graduate,1
4,0.528302,0.200000,0.192308,0.653422,0.0,0.26087,0.30,0.700000,0.732558,Graduate,1


In [3]:
feature_columns = data.columns.drop(['Target', 'y_labels'])
X = data[feature_columns].to_numpy(dtype=float)
y = data['y_labels'].to_numpy(dtype=int)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, shuffle=True)

In [5]:
# Polynomial Kernel
def polynomial_kernel(A, B, *, degree=3, gamma=1., coef0=1.):
    """
    K(x, x') = (gamma * <x, x'> + coef0)^degree
    """
    return (gamma * (A @ B.T) + coef0) ** degree

In [6]:
# Gaussian Kernel
def gaussian_kernel(A, B, *, gamma=1.):
    """
    K(x, x') = exp( - gamma * ||x - x'||^2 )
    """
    A2 = (A**2).sum(1)[:, None]
    B2 = (B**2).sum(1)[None, :]
    return np.exp(-gamma * (A2 + B2 - 2 * (A @ B.T)))

In [7]:
def train_kernel_svm_sgd(X, y, kernel, T=1000, C=1.0, **kernel_params):
    """
    Dual-SGD for hinge-loss SVM using a precomputed Gram matrix.
    Returns the averaged alpha coefficients.
    """
    n = X.shape[0]
    # precompute Gram matrix
    K = kernel(X, X, **kernel_params)
    
    # betas accumulate raw counts; we'll convert to alphas each step
    beta = np.zeros(n)
    alpha_sum = np.zeros(n)
    
    for t in range(1, T + 1):
        # compute current alphas from beta
        alpha = beta / (2 * C * t)
        
        # uniformly pick one example at random
        j = np.random.randint(n)
        
        # decision function margin for j
        margin_j = y[j] * np.dot(alpha * y, K[:, j])
        
        # if margin_j < 1, we incur hinge loss → update beta_j
        if margin_j < 1:
            beta[j] += y[j]
        
        alpha_sum += alpha
    
    # return the averaged alpha over all iterations
    return alpha_sum / T

In [8]:
def predict_kernel_svm(alpha, X_train, X_eval, kernel, **kernel_params):
    """
    Predict sign( f(x) ) where
      f(x) = sum_i alpha_i * K(x_i, x)
    """
    return np.sign(alpha @ kernel(X_train, X_eval, **kernel_params))

In [9]:
def cross_validate(params, X, y, kernel, *, k_folds=5, T=1000):
    """
    Perform k-fold cross-validation for kernel SVM with given parameters
    Returns average accuracy across folds
    """
    C = params['C']
    kernel_params = {}
    for key, val in params.items():
        if key != 'C':
            kernel_params[key] = val

    acc = []
    skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

    for train_idx, val_idx in skf.split(X, y):
        X_tr, X_val = X[train_idx], X[val_idx]
        y_tr, y_val = y[train_idx], y[val_idx]

        alpha = train_kernel_svm_sgd(X_tr, y_tr, kernel, C=C, T=T, **kernel_params)
        preds = predict_kernel_svm(alpha, X_tr, X_val, kernel, **kernel_params)

        acc.append((preds == y_val).mean())

    return np.mean(acc)

In [10]:
def grid_search_svm(X, y, kernel, param_grid, *, k_folds=5, T=1000, n_jobs=-1):
    """
    Perform grid search to find best hyperparameters
    Returns best parameters and best score
    """
    grid = list(ParameterGrid(param_grid))

    best_score = -float('inf')   # start lower than any real score
    best_params = None

    for params in grid:
        score = cross_validate(params, X, y, kernel, k_folds=k_folds, T=T)
        print(f"Tested {params} → CV accuracy = {score:.4f}")

        if score > best_score:
            best_score  = score
            best_params = params

    return best_params, best_score

In [11]:
# Hyperparameter grid for Polynomial kernel
poly_grid = dict(
    degree=[2, 3, 4],                         
    gamma=[1e-4, 1e-3, 1e-2, 0.1, 1],         
    coef0=[0, 0.5, 1, 2],                     
    C=[
        1e-4, 3e-4, 1e-3, 3e-3,               
        1e-2, 3e-2, 1e-1, 0.3,                
        1, 3, 10                              
    ]
)

In [13]:
# Hyperparameter grid for Gaussian kernel
gaussian_grid = dict(
    gamma=[
        1e-5, 3e-5, 1e-4, 3e-4,            
        1e-3, 3e-3, 1e-2, 3e-2,
        1e-1, 3e-1, 1, 3                   
    ],
    C=[
        1e-4, 3e-4, 1e-3, 3e-3,            
        1e-2, 3e-2, 1e-1, 0.3,             
        1, 3, 10                           
    ]
)

In [12]:
# Perform grid search for polynomial kernel
print(">> Polynomial grid-search")
best_poly, poly_cv = grid_search_svm(X_train, y_train, polynomial_kernel, poly_grid)
print("Best poly params:", best_poly, "CV =", poly_cv)

>> Polynomial grid-search
Tested {'C': 0.0001, 'coef0': 0, 'degree': 2, 'gamma': 0.0001} → CV accuracy = 0.6787
Tested {'C': 0.0001, 'coef0': 0, 'degree': 2, 'gamma': 0.001} → CV accuracy = 0.6793
Tested {'C': 0.0001, 'coef0': 0, 'degree': 2, 'gamma': 0.01} → CV accuracy = 0.6787
Tested {'C': 0.0001, 'coef0': 0, 'degree': 2, 'gamma': 0.1} → CV accuracy = 0.3213
Tested {'C': 0.0001, 'coef0': 0, 'degree': 2, 'gamma': 1} → CV accuracy = 0.3213
Tested {'C': 0.0001, 'coef0': 0, 'degree': 3, 'gamma': 0.0001} → CV accuracy = 0.6784
Tested {'C': 0.0001, 'coef0': 0, 'degree': 3, 'gamma': 0.001} → CV accuracy = 0.6787
Tested {'C': 0.0001, 'coef0': 0, 'degree': 3, 'gamma': 0.01} → CV accuracy = 0.6776
Tested {'C': 0.0001, 'coef0': 0, 'degree': 3, 'gamma': 0.1} → CV accuracy = 0.3213
Tested {'C': 0.0001, 'coef0': 0, 'degree': 3, 'gamma': 1} → CV accuracy = 0.3213
Tested {'C': 0.0001, 'coef0': 0, 'degree': 4, 'gamma': 0.0001} → CV accuracy = 0.6790
Tested {'C': 0.0001, 'coef0': 0, 'degree': 4, 'gam

In [14]:

# Perform grid search for Gaussian kernel
print("\n>> Gaussian grid-search")
best_gaussian, gaussian_cv = grid_search_svm(X_train, y_train, gaussian_kernel, gaussian_grid)
print("Best gaussian params:", best_gaussian, "CV =", gaussian_cv)


>> Gaussian grid-search
Tested {'C': 0.0001, 'gamma': 1e-05} → CV accuracy = 0.3213
Tested {'C': 0.0001, 'gamma': 3e-05} → CV accuracy = 0.3213
Tested {'C': 0.0001, 'gamma': 0.0001} → CV accuracy = 0.3213
Tested {'C': 0.0001, 'gamma': 0.0003} → CV accuracy = 0.3213
Tested {'C': 0.0001, 'gamma': 0.001} → CV accuracy = 0.3213
Tested {'C': 0.0001, 'gamma': 0.003} → CV accuracy = 0.3213
Tested {'C': 0.0001, 'gamma': 0.01} → CV accuracy = 0.3213
Tested {'C': 0.0001, 'gamma': 0.03} → CV accuracy = 0.3213
Tested {'C': 0.0001, 'gamma': 0.1} → CV accuracy = 0.3213
Tested {'C': 0.0001, 'gamma': 0.3} → CV accuracy = 0.3213
Tested {'C': 0.0001, 'gamma': 1} → CV accuracy = 0.3213
Tested {'C': 0.0001, 'gamma': 3} → CV accuracy = 0.3213
Tested {'C': 0.0003, 'gamma': 1e-05} → CV accuracy = 0.3213
Tested {'C': 0.0003, 'gamma': 3e-05} → CV accuracy = 0.3213
Tested {'C': 0.0003, 'gamma': 0.0001} → CV accuracy = 0.3213
Tested {'C': 0.0003, 'gamma': 0.0003} → CV accuracy = 0.3213
Tested {'C': 0.0003, 'gam

In [15]:
def fit_and_eval(best_params, kernel, name):
    C = best_params['C']
    kernel_params = {}
    for key, val in best_params.items():
        if key != 'C':
            kernel_params[key] = val

    alpha = train_kernel_svm_sgd(X_train, y_train, kernel, C=C, T=1000, **kernel_params)
    preds = predict_kernel_svm(alpha, X_train, X_test, kernel, **kernel_params)

    acc = accuracy_score(y_test, preds)
    print(f"\n{name} kernel - test accuracy: {acc:.4f}")


In [16]:
# Train Polynomial kernel model
fit_and_eval(best_poly, polynomial_kernel, "Polynomial")


Polynomial kernel - test accuracy: 0.7955


In [17]:
# Train Gaussian kernel model
fit_and_eval(best_gaussian, gaussian_kernel, "Gaussian")


Gaussian kernel - test accuracy: 0.8192


In [18]:
from sklearn.metrics import log_loss, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [24]:
def fit_and_eval_metrics(best_params, kernel, name):
    C = best_params['C']
    kernel_params = {}
    for key, val in best_params.items():
        if key != 'C':
            kernel_params[key] = val

    alpha = train_kernel_svm_sgd(X_train, y_train, kernel, C=C, T=1000, **kernel_params)
    
    # Compute decision scores (raw SVM output)
    decision_scores = alpha @ kernel(X_train, X_test, **kernel_params)
    
    # Predicted class labels
    preds = np.sign(decision_scores)
    
    # Probabilities for log-loss (sigmoid)
    probs = 1 / (1 + np.exp(-decision_scores))
    probs = np.vstack([1 - probs, probs]).T

    # True labels must be 0/1 for log-loss
    y_true_binary = (y_test + 1) // 2

    # Metrics
    acc = accuracy_score(y_test, preds)
    logloss = log_loss(y_true_binary, probs)
    precision = precision_score(y_test, preds)
    recall = recall_score(y_test, preds)
    f1 = f1_score(y_test, preds)
    cm = confusion_matrix(y_test, preds)

    # Print metrics
    print(f"\n{name} kernel - Test Metrics:")
    print(f"  Accuracy     : {acc:.4f}")
    print(f"  Log-Loss     : {logloss:.4f}")
    print(f"  Precision    : {precision:.4f}")
    print(f"  Recall       : {recall:.4f}")
    print(f"  F1 Score     : {f1:.4f}")
    print(f"  Confusion Matrix:\n{cm}")

In [25]:
fit_and_eval_metrics(best_poly, polynomial_kernel, "Polynomial")


Polynomial kernel - Test Metrics:
  Accuracy     : 0.7514
  Log-Loss     : 0.6647
  Precision    : 0.8202
  Recall       : 0.8120
  F1 Score     : 0.8161
  Confusion Matrix:
[[177 107]
 [113 488]]


In [26]:
fit_and_eval_metrics(best_gaussian, gaussian_kernel, "Gaussian")


Gaussian kernel - Test Metrics:
  Accuracy     : 0.8192
  Log-Loss     : 0.6156
  Precision    : 0.8155
  Recall       : 0.9484
  F1 Score     : 0.8769
  Confusion Matrix:
[[155 129]
 [ 31 570]]
